# DirectorScore 구하는 코드
- 필요 코드 : credits.csv, movies_cleaned_removed_onehot.csv, movies_metadata_date.csv
- 아웃풋 코드 : movies_cleaned_removed_onehot_director2.csv

In [1]:
import pandas as pd
import ast

In [2]:
cred = pd.read_csv('data/credits.csv')

print(cred.head())

                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862  


In [3]:
# crew column 안에 있는 job이 director인 사람의 이름과 id 뽑아내기
# crew column은 list of dictionary 형태로 되어있음

import ast

# Define a function to extract the director's name and id from the crew column
def extract_director_info(crew_data):
  crew_list = ast.literal_eval(crew_data)
  for member in crew_list:
    if member.get('job') == 'Director':
      return pd.Series([member.get('name'), member.get('id')])
  return pd.Series([None, None])

# Apply the function to extract director information and movie ID
cred[['director_name', 'director_id']] = cred['crew'].apply(extract_director_info)
director_df = cred[['director_name', 'director_id', 'id']]

# Drop rows where director information is missing
director_df.dropna(subset=['director_name', 'director_id'], inplace=True)

print(director_df.head())

     director_name  director_id     id
0    John Lasseter       7879.0    862
1     Joe Johnston       4945.0   8844
2    Howard Deutch      26502.0  15602
3  Forest Whitaker       2178.0  31357
4    Charles Shyer      56106.0  11862


/var/folders/_4/mvxsg_zj2q35755ky_zps0780000gn/T/ipykernel_47799/3864129096.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  director_df.dropna(subset=['director_name', 'director_id'], inplace=True)


In [4]:
# director_df 저장
director_df.to_csv('data/directors.csv', index=False)

In [5]:
result_df = pd.read_csv('data/movies_cleaned_removed_onehot.csv')


print(result_df.head())


      director_name  director_id     id    budget  popularity    revenue  \
0     John Lasseter       7879.0    862  30000000   21.946943  394436586   
1     Howard Deutch      26502.0  15602  25000000   11.712900   71500000   
2   Martin Campbell      10702.0    710  58000000   14.686036  352200000   
3    Steve Oedekerk       4489.0   9273  30000000    8.205448  212385533   
4  Barry Sonnenfeld       5174.0   8012  30250000   12.669608  115101622   

   runtime                           title  vote_average  vote_count  ...  \
0     81.0                       Toy Story           7.7      5415.0  ...   
1    101.0                Grumpier Old Men           6.5        92.0  ...   
2    130.0                       GoldenEye           6.6      1194.0  ...   
3     90.0  Ace Ventura: When Nature Calls           6.1      1128.0  ...   
4    105.0                      Get Shorty           6.4       305.0  ...   

   Family  Mystery  Romance  Animation Drama  Comedy  Foreign  Adventure  \
0   

In [6]:
# 저장
result_df.to_csv('data/movies_cleaned_removed_onehot_director.csv', index=False)

In [7]:
director_movies_dict = director_df.groupby('director_id')['id'].apply(list).to_dict()

# Define a function to get the list of movie IDs for each director in result_df
def get_director_movie_list(director_id):
    return director_movies_dict.get(director_id, [])

# Apply the function to add a new column to result_df with updated director information
result_df['director_movie_ids'] = result_df['director_id'].apply(get_director_movie_list)

print(result_df.head())


      director_name  director_id     id    budget  popularity    revenue  \
0     John Lasseter       7879.0    862  30000000   21.946943  394436586   
1     Howard Deutch      26502.0  15602  25000000   11.712900   71500000   
2   Martin Campbell      10702.0    710  58000000   14.686036  352200000   
3    Steve Oedekerk       4489.0   9273  30000000    8.205448  212385533   
4  Barry Sonnenfeld       5174.0   8012  30250000   12.669608  115101622   

   runtime                           title  vote_average  vote_count  ...  \
0     81.0                       Toy Story           7.7      5415.0  ...   
1    101.0                Grumpier Old Men           6.5        92.0  ...   
2    130.0                       GoldenEye           6.6      1194.0  ...   
3     90.0  Ace Ventura: When Nature Calls           6.1      1128.0  ...   
4    105.0                      Get Shorty           6.4       305.0  ...   

   Mystery  Romance  Animation  Drama Comedy  Foreign  Adventure  \
0        0  

In [10]:
metadata = pd.read_csv('data/movies_metadata_date.csv')

In [12]:
from datetime import datetime, timedelta

# Group director_df by director_id to create a dictionary of movie IDs
director_movies_dict = director_df.groupby('director_id')['id'].apply(list).to_dict()

# Update the result_df with director movie IDs
result_df['director_movie_ids'] = result_df['director_id'].apply(lambda x: director_movies_dict.get(x, []))

# Prepare the release date in movies_cleaned_df
metadata['release_year'] = metadata['release_year'].fillna(0).astype(int)
metadata['release_month'] = metadata['release_month'].fillna(1).astype(int)
metadata['release_day'] = metadata['release_day'].fillna(1).astype(int)

# Create a release date column in movies_cleaned_df
metadata['release_date'] = pd.to_datetime(
    metadata[['release_year', 'release_month', 'release_day']].rename(
        columns={'release_year': 'year', 'release_month': 'month', 'release_day': 'day'}
    ),
    errors = 'coerce'
)

# Create a dictionary to map movie IDs to release dates
movie_release_dates = metadata.set_index('id')['release_date'].to_dict()

# Define a function to filter movie IDs based on the release date within a specific range
def filter_recent_movies_within_range(row):
    movie_id = row['id']
    director_movies = row['director_movie_ids']
    
    # Get the release date of the current movie
    release_date = movie_release_dates.get(movie_id)
    
    if pd.isna(release_date):
        return []
    
    # Define the cutoff date as 3 years before the release date
    cutoff_date = release_date - timedelta(days=10*365)
    
    # Filter movies released within the range (cutoff_date, release_date - 1 day)
    recent_movies = [
        movie for movie in director_movies 
        if movie in movie_release_dates and cutoff_date < movie_release_dates[movie] < release_date
    ]
    
    return recent_movies

# Apply the function to filter the director_movie_ids in result_df
result_df['director_movie_ids'] = result_df.apply(filter_recent_movies_within_range, axis=1)

In [13]:
# director_movie_ids column 이름을 director_recent_movie_ids로 변경
result_df.rename(columns={'director_movie_ids': 'director_recent_movie_ids'}, inplace=True)

print(result_df.head())

      director_name  director_id     id    budget  popularity    revenue  \
0     John Lasseter       7879.0    862  30000000   21.946943  394436586   
1     Howard Deutch      26502.0  15602  25000000   11.712900   71500000   
2   Martin Campbell      10702.0    710  58000000   14.686036  352200000   
3    Steve Oedekerk       4489.0   9273  30000000    8.205448  212385533   
4  Barry Sonnenfeld       5174.0   8012  30250000   12.669608  115101622   

   runtime                           title  vote_average  vote_count  ...  \
0     81.0                       Toy Story           7.7      5415.0  ...   
1    101.0                Grumpier Old Men           6.5        92.0  ...   
2    130.0                       GoldenEye           6.6      1194.0  ...   
3     90.0  Ace Ventura: When Nature Calls           6.1      1128.0  ...   
4    105.0                      Get Shorty           6.4       305.0  ...   

   Mystery  Romance  Animation  Drama Comedy  Foreign  Adventure  \
0        0  

In [14]:
# director_recent_movie_ids 리스트의 원소 수 분포 구해보기 (value_counts)
result_df['director_recent_movie_ids'].apply(len).value_counts()


director_recent_movie_ids
0     286
1     216
2     188
3     177
4     125
5      96
6      58
7      34
8      21
9       5
15      2
10      2
24      1
27      1
17      1
11      1
13      1
16      1
12      1
Name: count, dtype: int64

In [17]:
# Create a dictionary to map movie IDs to revenue values
movie_revenue_dict = metadata.set_index('id')['revenue'].to_dict()

# Define a function to calculate the average revenue for director_recent_movie_ids
def calculate_average_revenue(row):
    movie_ids = row['director_recent_movie_ids']
    
    # Get the revenue values for the movie IDs
    revenues = [movie_revenue_dict.get(movie_id, 0) for movie_id in movie_ids if movie_revenue_dict.get(movie_id, 0) > 0]
    
    # Calculate the average revenue, excluding 0 values
    if len(revenues) > 0:
        return sum(revenues) / len(revenues)
    else:
        return 0

# Apply the function to calculate the average revenue and add it as a new column in result_df
result_df['average_director_movie_revenue'] = result_df.apply(calculate_average_revenue, axis=1)

In [21]:
#결과 값 정수로 반올림
result_df['average_director_movie_revenue'] = result_df['average_director_movie_revenue'].round().astype(int)

In [22]:
# 저장
result_df.to_csv('data/movies_cleaned_removed_onehot_director.csv', index=False)

In [23]:
# result_df에서 average_director_movie_revenue가 0인 값 갯수 출력
print(result_df[result_df['average_director_movie_revenue'] == 0].shape[0])

437


In [26]:
# result_df에서 average_director_movie_revenue가 0인데 director_recent_movie_ids가 있는 df 만들기
temp_df = result_df[(result_df['average_director_movie_revenue'] == 0) & (result_df['director_recent_movie_ids'].apply(len) > 0)]
temp_df.head()

,director_name,director_id,id,budget,popularity,revenue,runtime,title,vote_average,vote_count,...,Romance,Animation,Drama,Comedy,Foreign,Adventure,Science Fiction,Horror,director_recent_movie_ids,average_director_movie_revenue
6,Paul W.S. Anderson,4014.0,9312,18000000,10.870138,122195920,101.0,Mortal Kombat,5.4,452.0,...,0,0,0,0,0,0,0,0,[45549],0
37,Andrew Morahan,53599.0,8011,30000000,6.866634,36700000,99.0,Highlander: The Final Dimension,4.5,110.0,...,0,0,0,0,0,0,1,0,[20227],0
56,Danny Boyle,2034.0,627,4000000,19.348466,72000000,93.0,Trainspotting,7.8,2737.0,...,0,0,1,0,0,0,0,0,[9905],0
71,Brett Leonard,57090.0,10163,10000000,6.685948,32100816,108.0,The Lawnmower Man,5.4,198.0,...,0,0,0,0,0,0,1,1,[40448],0
82,Ridley Scott,578.0,348,11000000,23.377420,104931801,117.0,Alien,7.9,4564.0,...,0,0,0,0,0,0,1,1,[19067],0


- average_director_movie_revenue가 0인 행 갯수 : 437개
- 그 중 애초에 근 10년 간 제작한 영화가 없었던 감독의 영화 갯수 : 286개 -> 이 경우 average값 0
- 나머지 151개 처리 방법
  - average_director_movie_revenue 값이 존재하는 데이터들에 한해 vote_count별 평균 revenue값 구하기
  - direcotr_recent_movie_ids column에 있는 영화 번호들의 vote_count를 구하고 앞에 구해놓은 vote_count별 평균 revenue값을 이용해 적절한 revenue값 예상하기
  - 그 revenue값들의 평균으로 average_director_movie_revenue값 채우기

In [29]:
# Create a dictionary to map movie IDs to revenue and vote_count values
movie_revenue_dict = metadata.set_index('id')['revenue'].to_dict()
movie_vote_count_dict = metadata.set_index('id')['vote_count'].to_dict()

# Step 1: Calculate vote_count-based average revenue for movies with non-zero revenue
# Filter out movies with revenue > 0 and group by vote_count
valid_revenues_df = metadata[metadata['revenue'] > 0]
vote_count_avg_revenue = valid_revenues_df.groupby('vote_count')['revenue'].mean().to_dict()

# Define a function to handle missing average_director_movie_revenue for the 151 cases
def estimate_average_revenue(row):
    # If the average_director_movie_revenue is non-zero, keep it as is
    if row['average_director_movie_revenue'] > 0:
        return row['average_director_movie_revenue']
    
    # Get the director's recent movie IDs
    movie_ids = row['director_recent_movie_ids']
    
    # Get the vote_count for each movie and calculate an estimated revenue based on the average revenue per vote_count
    estimated_revenues = []
    for movie_id in movie_ids:
        vote_count = movie_vote_count_dict.get(movie_id)
        if vote_count in vote_count_avg_revenue:
            estimated_revenues.append(vote_count_avg_revenue[vote_count])
    
    # Calculate the average of estimated revenues if available
    if len(estimated_revenues) > 0:
        return sum(estimated_revenues) / len(estimated_revenues)
    else:
        return 0

# Apply the function to estimate the missing average_director_movie_revenue values
result_df['average_director_movie_revenue'] = result_df.apply(estimate_average_revenue, axis=1)

In [34]:
#average_director_movie_revenue 값 정수로 반올림
result_df['average_director_movie_revenue'] = result_df['average_director_movie_revenue'].round().astype(int)

In [35]:
# 결과 저장
result_df.to_csv('data/movies_cleaned_removed_onehot_director2.csv', index=False)


286
